In [1]:
#Imports
import pandas as pd
from datetime import datetime
from g2p_en import G2p
#pip install g2p_en <- erforderlich für die Phonemtranskribierung
#oder python setup.py install

In [2]:
#CSV einlesen und Dataframes erstellen
twistList = pd.read_csv(r'C:\Users\flosc\Documents\DHBW\VisualStudioCode\NLP\tongue_twisters\data source 2\raw-data\raw-data\twister_dataset.csv', delimiter=',',nrows= 3000,header=None)
kaggle = pd.read_csv(r'C:\Users\flosc\Documents\DHBW\VisualStudioCode\NLP\tongue_twisters\data source 2\raw-data\raw-data\datasetKaggle.csv', delimiter=',',nrows= 1000,header=None)
#print(kaggle)
#print(twistList)

In [3]:
#Beide DataFrames "zuschneiden"
dfTL = twistList.drop([2, 3], axis=1)
#print(dfTL)
dfKaggle = kaggle.drop([0], axis=1)
#print(dfKaggle)


In [4]:
#dfKaggle in Phoneme übersetzen
duplic = dfKaggle[~dfKaggle[1].isin(dfTL[0])] #<- nur die Zungenbrecher, welche noch nicht in der TwistList vorkommen
#print(duplic)
dfKaggleP = [] #<- leeres DataFrame initialisieren
g2p = G2p()
for index, row in duplic.iterrows():
    text = row[1]
    #print(text)
    dfKaggleP.append(g2p(text))
print(dfKaggleP)
#print(len(dfKaggleP))


[['Z', 'IH1', 'R', 'OW0'], ['T', 'UW1', ' ', 'S', 'IH1', 'T', ' ', 'IH0', 'N', ' ', 'S', 'AA1', 'L', 'AH0', 'M', ' ', 'S', 'AY1', 'L', 'AH0', 'N', 'S', ' ', 'IH0', 'N', ' ', 'AH0', ' ', 'D', 'AH1', 'L', ' ', ',', ' ', 'D', 'AA1', 'R', 'K', ' ', 'D', 'AA1', 'K', ' ', ',', ' ', 'IH0', 'N', ' ', 'AH0', ' ', 'P', 'EH2', 'S', 'T', 'IH0', 'T', 'EH1', 'N', 'SH', 'AH0', 'L', ' ', 'P', 'R', 'IH1', 'Z', 'AH0', 'N', ' ', ',', ' ', 'W', 'IH1', 'DH', ' ', 'AH0', ' ', 'L', 'AY1', 'F', 'L', 'AO0', 'NG', ' ', 'L', 'AA1', 'K', ' ', ',', ' ', 'AH0', 'W', 'EY1', 'T', 'IH0', 'NG', ' ', 'DH', 'AH0', ' ', 'S', 'EH0', 'N', 'S', 'EY1', 'SH', 'AH0', 'N', ' ', 'AH1', 'V', ' ', 'AH0', ' ', 'SH', 'AO1', 'R', 'T', ' ', ',', ' ', 'SH', 'AA1', 'R', 'P', ' ', 'SH', 'AA1', 'K', ' ', ',', ' ', 'F', 'R', 'AH1', 'M', ' ', 'AH0', ' ', 'CH', 'IY1', 'P', ' ', 'AH0', 'N', 'D', ' ', 'CH', 'IH1', 'P', 'IY0', ' ', 'CH', 'AA1', 'P', 'ER0', ' ', 'AA1', 'N', ' ', 'AH0', ' ', 'B', 'IH1', 'G', ' ', 'B', 'L', 'AE1', 'K', ' ', 'B', 'L

In [5]:
dfKaggleP = pd.DataFrame(dfKaggleP)
print(dfKaggleP)
#finKaggle = pd.merge(duplic, dfKaggleP, left_on=[0], right_on=[0], how='outer')
#finKaggle = pd.concat([duplic,dfKaggleP], axis=1)
#print(finKaggle)

    0    1   2    3     4     5     6     7     8     9    ...   695   696  \
0     Z  IH1   R  OW0  None  None  None  None  None  None  ...  None  None   
1     T  UW1        S   IH1     T         IH0     N        ...  None  None   
2   AY1        S    K     R   IY1     M           ,        ...  None  None   
3     S  EH1   L  IH0     B   AH0     T           S   EH1  ...  None  None   
4     W  AH1   T        AH0           T   EH1     R   AH0  ...  None  None   
..  ...  ...  ..  ...   ...   ...   ...   ...   ...   ...  ...   ...   ...   
70   DH  EH1   N          S     T   EH1     P         AH1  ...  None  None   
71    N  AW1        K   IH1     S   AH0     L           W  ...  None  None   
72    P  ER1   P  AH0     L           P   EY1     P   ER0  ...  None  None   
73    D  IY1        D   AO1     R     G   AH0     N     Z  ...  None  None   
74    N  OW1   P          ,         AE1     N         AE1  ...  None  None   

     697   698   699   700   701   702   703   704  
0   None  

In [ ]:
#Dataframes zusammenführen
#duplic = dfKaggleP[not dfKaggleP[1].isin(dfTL[0])]
#duplic = dfKaggleP[~dfKaggleP[1].applymap(lambda x: any(item in x for item in dfTL[0]))]
#print(duplic)
#finDF = pd.concat([dfTL,duplic.rename(columns={0:1,1:0})], ignore_index=True)
duplic['phonemes'] = dfKaggleP
#kaggleDF = pd.DataFrame(dfKaggleP, columns= [0])
#print(kaggleDF)
finDF = pd.concat([dfTL, dfKaggleP])
#finDF = finDF.drop(2628)
finDF=finDF[(finDF[0]!= 0) | (finDF[1] != 0)]
test = finDF[finDF[0] == 0]
print(test)
print(finDF)

In [ ]:
#Satzzeichen aus Phonemen entfernen
#finDF = finDF[1].str.replace(r'[\[!]]','',regex=True)
#print(finDF)

In [ ]:
#DataFrame als .csv exportieren
date = datetime.now().strftime('%Y-%m-%d')
filename = f'{date}_Dataset.csv'
finDF.to_csv(filename, index= False)